# Generalizations of Max-Flow

The purpose of this assignment is to learn about the min-cost flow problem, a generalization of max-flow, and to familiarize yourself with implementing and solving linear programs.

## Min-Cost Flow

Recall that a flow network with demands consists of a directed graph $G = (V, E)$, where each edge $(i,j) \in E$ has a positive integer capacity $c_{ij}$ and each node $i \in V$ has an integer demand $d_i$. In a *min-cost flow* problem, each edge $(i,j) \in E$ also has a cost (or weight) $w_{ij}$. (Note that this input generalizes the input to two important problems we discussed so far: in max flow, the edge weights were not important while in shortest paths, the edge capacities were not important.) 

Given a flow network with capacities and costs, the goal is to find a *feasible* flow $f: E \rightarrow R^+$ --that is, a flow satisfying edge capacity constraints and node demands-- that minimizes the total cost of the flow. Explicitly, the problem can be formulated as a linear program.

### Question 1

Answer Problem 1 in HW4-theoretical.

### Question 2

To implement your reduction from Problem 1 in HW4-theoretical, you will work with some standard benchmark instances for min-cost flow found [here](http://elib.zib.de/pub/Packages/mp-testdata/mincost/gte/index.html). The format of the data is described in the [Info](http://elib.zib.de/pub/Packages/mp-testdata/mincost/gte/info) file. You are to read in the graph from the data file in a form that can be solved using NetworkX's `min_cost_flow` function. Note that the data sometimes lists multiple edges between the same nodes, but with different costs or capacities. In forming the graph, you need to implement your reduction from the previous question and form a `DiGraph` instance, because the `min_cost_flow` function cannot handle multi-edges, even though the package offers `MultiDiGraph` objects.

In [1]:
import networkx as nx

def create_graph(infile):
    """Creates a directed graph as specified by the input file. Edges are annotated with 'capacity'
    and 'weight' attributes, and nodes are annotated with 'demand' attributes.
    
    Args:
        infile: the input file using the format to specify a min-cost flow problem.
        
    Returns:
        A directed graph (but not a multi-graph) with edges annotated with 'capacity' and 'weight' attributes
        and nodes annotated with 'demand' attributes.
    """
    # TODO: implement function
    myG=nx.DiGraph()
    myfile=open(infile,'r')
    while True:
        line = myfile.readline().strip()
        if line == "":
            break
        if line[0] == 'p':                  
            n = int(line.split()[2])
            myG.add_nodes_from(range(1, n+1), demand = 0)
            exist = [[0 for j in range(n)] for i in range(n)]
        elif line[0] == 'n':
            myG.node[int(line.split()[1])]['demand'] = int(line.split()[2])
        elif line[0] == 'a':
            line = [int(x) for x in line.split()[1:]]
            if exist[line[0]-1][line[1]-1] == 0: 
                myG.add_edge(line[0], line[1], capacity = line[3], weight = line[4])
                exist[line[0]-1][line[1]-1] = 1
            else:
                n = n+1
                myG.add_node(n, demand = 0)
                myG.add_edge(line[0], n, capacity = line[3], weight = line[4])
                myG.add_edge(n, line[1], capacity = line[3], weight = 0)
    return myG

The following will check that your code outputs the expected min cost flow values on several test instances.

In [2]:
G_40 = create_graph('gte_bad.40')
G_6830 = create_graph('gte_bad.6830')
G_176280 = create_graph('gte_bad.176280')

print "Correct value for _40 instance:", nx.min_cost_flow_cost(G_40) == 52099553858
print "Correct value for _6830 instance:", nx.min_cost_flow_cost(G_6830) == 299390431788
print "Correct value for _176280 instance:", nx.min_cost_flow_cost(G_176280) == 510585093810

Correct value for _40 instance: True
Correct value for _6830 instance: True
Correct value for _176280 instance: True


## Linear Programming

Instead of using special-purpose min-cost flow solvers, you will now formulate the problems as linear programs and use general-purpose LP solvers to find the solutions.

### Question 3

Implement the following function to formulate the flow LP and return the optimal value (i.e., minimum cost over feasible flows).

In [3]:
import pulp

def lp_flow_value(G):
    """Computes the value of the minimum cost flow by formulating and solving
    the problem as an LP.
    
    Args:
        G: a directed graph with edges annotated with 'capacity' and 'weight'
            attrbutes, and nodes annotated with 'demand' attributes.
            
    Returns:
        The value of the minimum cost flow.
    """
    # TODO: implement function
    prob=pulp.LpProblem("mincostflow", pulp.LpMinimize)
    capacity = nx.get_edge_attributes(G, 'capacity')
    weight = nx.get_edge_attributes(G, 'weight')
    demand = nx.get_node_attributes(G, 'demand')
    flow = pulp.LpVariable.dicts('flow', G.edges(), 0, None, pulp.LpInteger)
    prob += sum([weight[edge]*flow[edge] for edge in G.edges()]), "Total Cost of Flow"
    for node in G.nodes():
        prob += sum(flow[in_edge] for in_edge in G.in_edges(node)) - sum(flow[out_edge] for out_edge in G.out_edges(node)) == demand[node]
    for edge in G.edges():
        prob += flow[edge] <= capacity[edge]
    prob.solve()
    return int(pulp.value(prob.objective))

In [4]:
print "Correct value for _40 instance:", lp_flow_value(G_40) == 52099553858
print "Correct value for _6830 instance:", lp_flow_value(G_6830) == 299390431788
print "Correct value for _176280 instance:", lp_flow_value(G_176280) == 510585093810

Correct value for _40 instance: True
Correct value for _6830 instance: True
Correct value for _176280 instance: True
